# Chapter 4.6. - D-Wave Advantage and Advantage2

**Dann5 d5o programming framework** is an extension of python programming langue. By generalizing D-Wave CSP implementation, it helps QAC programmers develop faster programming logic through use of constructs such as data types definitions, equation and condition statements, programmable routines, functions, and specialized functions.

Here, we are continuing an overview of d5o features by:
1.	Introducing **d5o Qsolverand Qanalyzer**. Qsolver is a local Qubo solver, an alternative to [D-Wave’s ExactSolver]( https://docs.ocean.dwavesys.com/projects/dimod/en/latest/reference/sampler_composites/samplers.html#module-dimod.reference.samplers.exact_solver).
2.	executing complex d5o equations on QAC using [D-Wave samplers]( https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html#dwavesampler); 
3.	solving a large complex d5o equation using [D-Wave hybrid-sampler]( https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html#leaphybridsampler);


If you have [installed Dann5 d5o library](https://github.com/voya-voja/dann5/blob/master/README.md) and all prerequisites, we can start.

In [ ]:
import dann5.d5 as d5
from dann5.dwave import Solver
p = d5.Qwhole(3,"p")
q = d5.Qwhole(2, "q")
r = d5.Qwhole(2, "r")
Sum = d5.Qwhole("S", 10)
sumAssignment = Sum.assign(p + q + r)
print(sumAssignment)

Also, the same problem can be solved using **Dwave Advantage quantum annealer** as it is presented in the following example.

In [ ]:
from dann5.dwave import QuboSolvers

hybrid = QuboSolvers.solver("Advantage")
evaluations = hybrid.solution(qubo)

sumAssignment.reset()
sumAssignment.add(evaluations)
print("DWave Advantage QUBO solutions: \n{}".format(sumAssignment.solutions()))

> Important to notice that the quantum annealer will require additional operational arguments specified in [**kwargs**](https://docs.dwavesys.com/docs/latest/c_solver_parameters.html).
- **Note**: This example demonstraits setting up different DWave sampler topology types. It will be discussed further in chapter 5 as an advanced features.

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus', 'qpu': True})
kwargs = {'num_reads': 1000,
            'answer_mode': 'histogram',
            'chain_strength': 2  
        }
advantage = EmbeddingComposite(qpu_advantage)
sampleset = advantage.sample_qubo(qubo, **kwargs)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
print("DWave Advantage solutions: \n{}".format(samples))


The same **finalized QUBO** can be executed on [quantum annealing computer (QAC)](https://docs.dwavesys.com/docs/latest/c_solver_parameters.html):

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus', 'qpu': True})
solver = EmbeddingComposite(qpu_advantage)
sampleset = solver.sample_qubo(fQaA)

In [ ]:
samples = [dict(sample) for sample in sampleset.lowest().samples()]
aA.reset()
aA.add(samples)
print(aA.solutions())

In [1]:
from dann5.d5o import Qvar, Qequation, Qanalyzer
import time

ImportError: generic_type: type "QuboCompiler" referenced unknown base type "dann5::Qcompiler"

For a problem:

> If **T = 2 * a + b * c**, find all possible solutions when **T = 11** and **a, b, c <= 3**.

Like in the [d5o overview of basic features](https://github.com/voya-voja/dann5/blob/master/examples/py/d5o_basic_features.ipynb), we are using python variable **_2**, **a**, **b**, **c** and **T** to reference *Q variables* ***2_***, ***a***, ***b***, ***c*** and ***T*** respectfully, where 
- Q constant *2_* has a value **2**; 
- Q variables *a*, *b* and *c* are initialized with **2 Q bits**, i.e. limiting their values to {0, 1, 2, 3}
- Q variable *T* is set to **11**, and it is used as a Q equation result.

Python variable **eT** references the *Q equation* **T = 2_ * a + b * c**

***Note:*** parentheses around multipliers are required in python implementation, *(still to investigate why)*. In C++ implementation they are not required as *multiplication operator has precedence over addition operator*.

In [2]:
_2 = Qvar("2_",2)
a = Qvar(2, "a")
b = Qvar(2, "b")
c = Qvar(2, "c")
T = Qvar("T", 11)
eT = Qequation(T)
eT.assign((_2 * a) + (b * c))
print(eT.toString())

| T0 = 1 | = 2_0 & a0 ^ b0 & c0
| T1 = 1 | = 2_0 & a1 + 2_1 & a0 + b0 & c1 ^ b1 & c0 ^ #|T0|
| T2 = 0 | = 2_1 & a1 + #|2_0 & a1 + 2_1 & a0 + b0 & c1 ^ b1 & c0| + b1 & c1 ^ #|b0 & c1 ^ b1 & c0| ^ #|T1|
| T3 = 1 | = #|2_1 & a1 + #|2_0 & a1 + 2_1 & a0 + b0 & c1 ^ b1 & c0| + b1 & c1 ^ #|b0 & c1 ^ b1 & c0|| + #|b1 & c1 ^ #|b0 & c1 ^ b1 & c0|| + #|T2|
| T4 = 0 | = #|T3|



To better understand a complexity of the Q equation above, we are analyzing the Qubo generated for the 

In [3]:
Q = eT.qubo()
analyzer = Qanalyzer(Q)
nNo = analyzer.nodesNo()
print("number of nodes:", nNo, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 25 has 8,388,658.0 possible solutions


By using d5o we can easily solve on a local machine a problem like:
> A school has received 45 tickets for a show. If there are 13 schoolteachers to oversee pupils, with a help of no more than 3 parents, what would be a similar number of boys and girls that can see the show?

In [8]:
girls = Qvar(4, "girls")
boys = Qvar(4, "boys")
teachers = Qvar("teachers", 13)
parents = Qvar(2, "parents")
tickets = Qvar("tickets", 45)
distribution = Qequation(tickets).assign( girls + boys + teachers + parents )
Q = distribution.qubo()
nNo = Qanalyzer(Q).nodesNo()
print("number of nodes:", nNo, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 28 has 67,108,920.0 posible solutions


## d5o equations on QAC using D-Wave samplers

When defining Q variables to solve a problem, it is important to ensure that there is a possible solution within defined Q equation. For example, 
> if we would like *to rerun the above school trip example for 78 available tickets*, **and we do not adjust variables**, ***we will get a result set that does not make any sense***:

In [14]:
tickets = Qvar("tickets", 78)
distribution = Qequation(tickets).assign( girls + boys + teachers + parents )
Q = distribution.qubo()

girls = 0; boys = 0; teachers = 13; parents = 1; 
girls = 1; boys = 0; teachers = 13; parents = 0; 
girls = 0; boys = 1; teachers = 13; parents = 0; 



Obviously we need to adjust *the size of ***girls and boys*** Q variables*! However, this will result in Qubo with 33 nodes. 

In [15]:
girls = Qvar(5, "girls")
boys = Qvar(5, "boys")
teachers = Qvar("teachers", 13)
parents = Qvar(2, "parents")
tickets = Qvar("tickets", 78)
distribution = Qequation(tickets).assign( girls + boys + teachers + parents )
Q = distribution.qubo()
nNo = Qanalyzer(Q).nodesNo()
print("number of nodes:", nNo`, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 33 has 2,147,483,714.0 posible solutions


> To avoid waiting so long in the future, I will use D-Wave Ocean sampler API to conenct to QACs to process my problem.

In [18]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.cloud.exceptions import SolverNotFoundError

To learn more about how to work with [different topologies](https://docs.ocean.dwavesys.com/en/stable/examples/topology_samplers.html) review D-Wave documentation. In this example we are connecting to both:
- advanatage QAC instance with pegasus topology, and
- 2000Q QAC instance with chimera topology

In [19]:
try:
    qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus', 'qpu': True})
    qpu_2000q = DWaveSampler(solver={'topology__type': 'chimera', 'qpu': True})
    
    qpus = {'Advantage': qpu_advantage, 'DW-2000Q': qpu_2000q}

    print("Connected to Advantage {} and 2000Q {}.".format(qpu_advantage.solver.id, qpu_2000q.solver.id))
except SolverNotFoundError:
    print("Currently a pair of solvers are unavailable for sections comparing QPU technologies. Try those examples later.")

Connected to Advantage Advantage_system1.1 and 2000Q DW_2000Q_6.


Now that we are connected to Advantage and 2000Q QACs, we can start by crating ***advantage*** solver by using D-Wave's composite [embedding](https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/embedding.html) of our Qubo to pegasus topology.

In [20]:
advantage = EmbeddingComposite(qpu_advantage)

For the ***advantage*** solver let prepare arguments for remote processing, i.e. **let set** ***kwargs***.
- ***kwargs*** *represents optional keyword arguments for the sampling method, specified per [solver](https://docs.dwavesys.com/docs/latest/doc_solver_ref.html) in parameters*. 

The arguments used in this example are self explanatory.

> I am founding that ***chain strenght 5*** works well for these kind of problems on pegasus topology.

In [21]:
kwargs = {}
if 'num_reads' in advantage.parameters:
    kwargs['num_reads'] = 100
if 'answer_mode' in advantage.parameters:
    kwargs['answer_mode'] = 'histogram'
if 'chain_strength' in advantage.parameters:
    kwargs['chain_strength'] = 5

In [22]:
start = time.process_time()
sampleset = advantage.sample_qubo(Q, **kwargs)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 0.375 seconds to solve problem with 33 nodes


In [23]:
distribution.set(sample)
print(distribution.solutions())

girls = 31; boys = 31; teachers = 13; parents = 3; 



Running *same kwargs* on **2000Q chimera topology**.

In [24]:
start = time.process_time()
dw2000q = EmbeddingComposite(qpu_2000q)
sampleset = dw2000q.sample_qubo(Q, **kwargs)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 0.296875 seconds to solve problem with 33 nodes


In [25]:
distribution.set(sample)
print(distribution.solutions())

girls = 31; boys = 31; teachers = 13; parents = 3; 



> The same ***chain strenght 5*** produced correct result on **chimera topology** as it did on *pegasus*.

As expected, on both QAC topologies it took a fraction of a second with all latencies of a remote API to solve a Qubo problem with 33 nodes.

In [32]:
factor1 = Qvar(5, "f1")
factor2 = Qvar(5, "f2")
factor3 = Qvar(4, "f3")
product = Qvar("P", 13020)
eProduct = Qequation(product).assign(factor1 * factor2 * factor3)
Q = eProduct.qubo()
nNo = Qanalyzer(Q).nodesNo()
print("number of nodes:", nNo, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 165 has 1.1692013098647223e+49 posible solutions


In [33]:
start = time.process_time()
sampleset = advantage.sample_qubo(Q, **kwargs)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 9.078125 seconds to solve problem with 165 nodes


In [34]:
eProduct.set(sample)
print(eProduct.solutions())

f1 = 0; f2 = 0; f3 = 0; 

